In [100]:
import pandas as pd
import numpy as np

from xgboost import XGBClassifier

from sklearn.metrics import accuracy_score,classification_report,recall_score,precision_score,roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV

In [2]:
df=pd.read_csv('wine.csv')

In [3]:
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [11]:
df['quality'].value_counts().to_dict()

{5: 681, 6: 638, 7: 199, 4: 53, 8: 18, 3: 10}

In [12]:
df['quality'].replace({5: 2, 6: 3, 7: 4, 4: 1, 8: 5, 3: 0},inplace=True)

In [13]:
x=df.drop('quality',axis=1)
y=df['quality']

In [29]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,stratify=y,random_state=50)

# XGBClassifier

In [71]:
model_xg=XGBClassifier()
model_xg.fit(x_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, objective='multi:softprob',
              predictor='auto', random_state=0, reg_alpha=0, ...)

In [72]:
y_predict_test=model_xg.predict(x_test)

accuracy_score1=accuracy_score(y_test,y_predict_test)
classification_report1=classification_report(y_test,y_predict_test)

print('Testing Data Evaluation of Random forest')
print(f'accuracy_score1: {accuracy_score1}')
print(classification_report1)

print('x'*60)

y_predict_train=model_xg.predict(x_train)

accuracy_score2=accuracy_score(y_train,y_predict_train)
classification_report2=classification_report(y_train,y_predict_train)

print('Training Data Evaluation of Random forest')
print(f'accuracy_score: {accuracy_score2}')
print(classification_report2)

print('x'*60)
print(f'Bias={abs(0.85-accuracy_score2)*100}')
print(f'Varience={(accuracy_score2-accuracy_score1)*100}')


Testing Data Evaluation of Random forest
accuracy_score1: 0.7020833333333333
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         3
           1       0.25      0.06      0.10        16
           2       0.74      0.81      0.77       204
           3       0.67      0.71      0.69       192
           4       0.73      0.55      0.63        60
           5       0.25      0.20      0.22         5

    accuracy                           0.70       480
   macro avg       0.44      0.39      0.40       480
weighted avg       0.69      0.70      0.69       480

xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
Training Data Evaluation of Random forest
accuracy_score: 1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         7
           1       1.00      1.00      1.00        37
           2       1.00      1.00      1.00       477
           3       1.00      1.00      1

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [73]:
model_xg.score(x_train,y_train)

1.0

# Hyper Parameter 

In [108]:
hyp={'n_estimators':np.arange(10,175),
    'max_depth':np.arange(5,12),
     'learning_rate':[0.01] 
    }

In [109]:
model_RSCV=RandomizedSearchCV(model_xg,hyp,cv=5)
model_RSCV.fit(x_train,y_train)

RandomizedSearchCV(cv=5,
                   estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                           callbacks=None, colsample_bylevel=1,
                                           colsample_bynode=1,
                                           colsample_bytree=1,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric=None, gamma=0, gpu_id=-1,
                                           grow_policy='depthwise',
                                           importance_type=None,
                                           interaction_constraints='',
                                           learning_rate=0.300000012,
                                           max_bin=256,...
        75,  76,  77,  78,  79,  80,  81,  82,  83,  84,  85,  86,  87,
        88,  89,  90,  91,  92,  93,  94,  95,  96,  97,  98,  99,

In [110]:
model_RSCV.best_estimator_

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.0001, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=10, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=158,
              n_jobs=0, num_parallel_tree=1, objective='multi:softprob',
              predictor='auto', random_state=0, reg_alpha=0, ...)

In [111]:
y_predict_test=model_RSCV.predict(x_test)

accuracy_score1=accuracy_score(y_test,y_predict_test)
classification_report1=classification_report(y_test,y_predict_test)

print('Testing Data Evaluation of Random forest')
print(f'accuracy_score1: {accuracy_score1}')
print(classification_report1)

print('x'*60)

y_predict_train=model_RSCV.predict(x_train)

accuracy_score2=accuracy_score(y_train,y_predict_train)
classification_report2=classification_report(y_train,y_predict_train)

print('Training Data Evaluation of Random forest')
print(f'accuracy_score: {accuracy_score2}')
print(classification_report2)

print('x'*60)
print(f'Bias={abs(0.85-accuracy_score2)*100}')
print(f'Varience={(accuracy_score2-accuracy_score1)*100}')


Testing Data Evaluation of Random forest
accuracy_score1: 0.6458333333333334
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         3
           1       0.14      0.06      0.09        16
           2       0.71      0.74      0.72       204
           3       0.62      0.66      0.64       192
           4       0.55      0.53      0.54        60
           5       1.00      0.20      0.33         5

    accuracy                           0.65       480
   macro avg       0.50      0.36      0.39       480
weighted avg       0.63      0.65      0.64       480

xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
Training Data Evaluation of Random forest
accuracy_score: 0.8436103663985701
              precision    recall  f1-score   support

           0       1.00      0.43      0.60         7
           1       1.00      0.57      0.72        37
           2       0.84      0.89      0.86       477
           3       0.83  

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
adj_r2score=1-(((1-r2score)(len(x_train)-1))/(len(x_train))-(len(x.columns))-(1))